In [1]:
import sys,os
sys.path.append("..")
import django
django.setup()
from django.conf import settings
from tw_data.stock_crawlers import *
from tw_data.future_crawlers import *
from tw_data.models import *
from etl.import_sql import *
import logging
import datetime

logging.basicConfig(level=logging.INFO)


# 台股股價

In [ ]:
C1=CrawlerProcess(StockPriceCrawlerTW,'crawl_main' ,StockPrice,'date_range')
logging.info(C1)

# # 指定日期區間爬蟲
# C1.specified_date_crawl('2020-9-22','2020-9-22')

# 自動爬蟲
C1.auto_update_crawl()

# 匯入課程股價資料

In [ ]:
class_df=pd.read_pickle('/Users/benbilly3/finlab/python_ai_stock_online_class/history/tables/price.pkl')
class_df=class_df.reset_index()
class_df['stock_name']=class_df['stock_id'].apply(lambda s:s[s.index(' ')+1:])
class_df['stock_id']=class_df['stock_id'].apply(lambda s:s[:s.index(' ')])
class_df=class_df[~class_df['stock_id'].isin(list(set([i for i in class_df['stock_id'].values if len(i)>4 and i[0]=='7'])))]

class_df= class_df.rename(columns={"證券代號": "stock_id", "證券名稱": "stock_name",
                                "成交股數": "vol", "成交金額": "turnover_price",
                                "開盤價": "open", "收盤價": "close",
                                "最高價": "high", "最低價": "low","成交筆數":"transactions_number",
                                "最後揭示買價":"finally_reveal_buy_price","最後揭示賣價":"finally_reveal_sell_price"})
class_df.iloc[:, 3:-1] =class_df.iloc[:, 3:-1].apply(lambda s: pd.to_numeric(s, errors="coerce"))
class_df

In [ ]:
class_df.to_pickle('/Users/benbilly3/finlab/finlab_data_center/finlab_data_center/static/pickle_table/tw_data/stock_price.pkl')

In [ ]:
class_df[class_df['stock_id']=='00731']

In [ ]:
df_test=class_df.head(500)
df_test

In [ ]:
def batchly_import_df(df,model,range_:int):
    total=len(df)+range_
    num_list=[i for i in range(0,total,range_)]
    for s,e in zip(num_list[:-1],num_list[1:]):
        print(s,e)
        sub_df=df.iloc[s:e]
        SqlImporter.add_to_sql(model, sub_df,check_date_rule=False)

batchly_import_df(class_df,StockPrice,200000)


In [ ]:
pp=StockPrice.objects.filter(stock_id='2330').values_list('date','close')

In [ ]:
import matplotlib.pyplot as plt
df_p=pd.DataFrame(list(pp))
df_p=df_p.set_index(0)
df_p.plot()

# 台股月營收

In [ ]:
C2=CrawlerProcess(MonthlyRevnueCrawlerTW,'crawl_main',MonthlyRevenue,'month_range')
print(C2)
# C2.specified_date_crawl('2015-5-10','2015-6-10')
C2.auto_update_crawl(deadline=15) 

# 集保餘額

In [2]:
C3=StockTdccCrawlerTW()
C3.crawl_main()
SqlImporter.add_to_sql(StockTdcc, C3.crawl_main(),directly_create=1)

INFO:etl.import_sql:Finish!<class 'tw_data.models.StockTdcc'> date:from 2020-12-25 to 2020-12-25 bulk_create:45040
INFO:etl.import_sql:Finish!<class 'tw_data.models.StockTdcc'> date:from 2020-12-25 to 2020-12-25 bulk_update:0


"Finish!<class 'tw_data.models.StockTdcc'> date:from 2020-12-25 to 2020-12-25 bulk_create:45040,Finish!<class 'tw_data.models.StockTdcc'> date:from 2020-12-25 to 2020-12-25 bulk_update:0"

# 台股企業基本資訊

In [ ]:
C4=CompanyInfoCrawlerTW()
SqlImporter.add_to_sql(CompanyInfo, C4.crawl_main())

# 台股指數點值資訊

In [ ]:
C5=CrawlerProcess(StockIndexPriceCrawlerTW,'crawl_main' ,StockIndexPrice,'date_range')
logging.info(C5)

# # 指定日期區間爬蟲
# C1.specified_date_crawl('2020-9-22','2020-9-22')

# 自動爬蟲
C5.auto_update_crawl()

# 台股指數成交量暨統計資訊

In [ ]:
C6=CrawlerProcess(StockIndexVolCrawlerTW,'crawl_main' ,StockIndexVol,'date_range')
logging.info(C6)

# # 指定日期區間爬蟲
# C1.specified_date_crawl('2020-9-22','2020-9-22')

# 自動爬蟲
C6.auto_update_crawl()

# 台股券商資訊

In [ ]:
C7=BrokerInfoCrawler()
SqlImporter.add_to_sql(BrokerInfo, C7.crawl_main())

# 台股法人買賣超現貨明細

In [ ]:
C8=CrawlerProcess(StockTiiCrawlerTW,'crawl_main' ,StockTii,'date_range')
logging.info(C8)

# # 指定日期區間爬蟲
# C1.specified_date_crawl('2020-9-22','2020-9-22')

# 自動爬蟲
C8.auto_update_crawl()

# 台股法人買賣超現貨日統計

In [ ]:
C9=CrawlerProcess(StockTiiMarketCrawlerTW,'crawl_main' ,StockTiiMarket,'date_range')
logging.info(C9)

# # 指定日期區間爬蟲
# C1.specified_date_crawl('2020-9-22','2020-9-22')

# 自動爬蟲
C9.auto_update_crawl()

# 融資交易明細

In [ ]:
C10=CrawlerProcess(StockMarginTransactionsCrawlerTW,'crawl_main' ,StockMarginTransactions,'date_range')
logging.info(C10)

# # 指定日期區間爬蟲
# C10.specified_date_crawl('2020-9-22','2020-9-22')

# 自動爬蟲
C10.auto_update_crawl()

# 本益比

In [ ]:
C11=CrawlerProcess(Stock3PRatioCrawlerTW,'crawl_main' ,Stock3PRatio,'date_range')
logging.info(C11)

# # 指定日期區間爬蟲
# C1.specified_date_crawl('2020-9-22','2020-9-22')

# 自動爬蟲
C11.auto_update_crawl()

# 除權息還原

In [ ]:
C12=StockDivideRatioCrawlerTW()
SqlImporter.add_to_sql(StockDivideRatio, C12.crawl_main())

# 股票期貨商品資訊

In [ ]:
C13=CommodityTaifexCrawlerTW()
SqlImporter.add_to_sql(StockListTaifex, C13.stock_relate(),fk_columns=[{'stock_id':'spot_id'}])

# 興櫃股價

In [ ]:
C14=CrawlerProcess(StockPriceCrawlerTW,'crawl_rotc' ,RotcStockPrice,'date_range')
logging.info(C4)

# # 指定日期區間爬蟲
# C1.specified_date_crawl('2007-1-4','2007-1-5')

# 自動爬蟲
C14.auto_update_crawl()

In [ ]:
dd=month_range(datetime.datetime(2020,11,10))
# dd=[i for i in dd if datetime.datetime.now() > i]
# dd